In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from utils.data import *
from utils.metrics import MAPE, WAPE, MAE
from utils.dl import *

In [2]:
dataset_path = "data/huge_stock_market_dataset/"

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

lags = 32
horizon = 8
stride = 1
batch_size = 256
val_size = 0.0
test_size = 0.3
drop_last = False
features = 1
keep_hs = False  # stride == lags
epochs = 200
verbose = False

model_params = {'input_size': features, 'hidden_size': 256, 'num_layers': 2, 'dropout': 0.1, 'output_size': horizon, 'seq_len': lags}

cuda:0


In [4]:
ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected.csv")

results = []
for time_series in tqdm(ts_iterator):
    train_dl, val_dl, test_dl, X_scaler, y_scaler = create_ts_dl(time_series[["Close"]], time_series["Close"], lags=lags, horizon=horizon, stride=stride,\
                                            batch_size=batch_size, device=device, data_preprocess=("log_returns", "normalize"),\
                                            val_size=val_size, test_size=test_size, drop_last=drop_last)
    
    model = RNNModel(seed=0, device=device)
    model.set_model(SimpleLSTM, **model_params)
    optim_params = {'params': model.model.parameters(), 'lr': 4e-4}
    model.set_optim(torch.optim.AdamW, **optim_params)
    model.set_criterion(MAE)

    model.train(train_dl, epochs=epochs, print_info=verbose, keep_hs=keep_hs, agg_loss="mean")
    results.append({"train": model.eval(train_dl, agg_loss="mean"), "test": model.eval(test_dl, agg_loss="mean")})

    del model, train_dl, val_dl, test_dl
    torch.cuda.empty_cache()
results

0it [00:00, ?it/s]Global seed set to 0
1it [00:11, 11.35s/it]Global seed set to 0
2it [00:23, 12.04s/it]Global seed set to 0
3it [00:36, 12.17s/it]Global seed set to 0
4it [00:46, 11.34s/it]Global seed set to 0
5it [00:54, 10.22s/it]Global seed set to 0
6it [01:18, 14.83s/it]Global seed set to 0
7it [01:36, 15.87s/it]Global seed set to 0
8it [01:56, 17.22s/it]Global seed set to 0
9it [02:22, 19.89s/it]Global seed set to 0
10it [02:43, 20.21s/it]Global seed set to 0
11it [03:22, 26.11s/it]Global seed set to 0
12it [04:04, 30.86s/it]Global seed set to 0
13it [04:47, 34.47s/it]Global seed set to 0
14it [05:21, 34.46s/it]Global seed set to 0
15it [06:03, 36.85s/it]Global seed set to 0
16it [06:49, 39.49s/it]Global seed set to 0
17it [07:39, 42.70s/it]Global seed set to 0
18it [08:26, 44.05s/it]Global seed set to 0
19it [09:13, 44.82s/it]Global seed set to 0
20it [10:00, 45.33s/it]Global seed set to 0
21it [12:47, 81.96s/it]Global seed set to 0
22it [15:45, 110.69s/it]Global seed set to 0
2

[{'train': 0.09766375925391912, 'test': 0.6188129186630249},
 {'train': 0.17890394727389017, 'test': 0.8885971307754517},
 {'train': 0.16966445123155913, 'test': 0.7931615114212036},
 {'train': 0.20025575160980225, 'test': 0.7587435245513916},
 {'train': 0.1953752264380455, 'test': 0.6375768780708313},
 {'train': 0.1174140989780426, 'test': 0.9096897840499878},
 {'train': 0.1769267227500677, 'test': 1.065084457397461},
 {'train': 0.1471579559147358, 'test': 0.6278383433818817},
 {'train': 0.09980894178152085, 'test': 0.7292180061340332},
 {'train': 0.1603152398020029, 'test': 0.9765946269035339},
 {'train': 0.12124018464237452, 'test': 0.765368640422821},
 {'train': 0.1958171036094427, 'test': 0.5109000727534294},
 {'train': 0.16982103697955608, 'test': 0.7010360658168793},
 {'train': 0.22042101515190943, 'test': 0.5724342962106069},
 {'train': 0.22555934637784958, 'test': 0.5533659160137177},
 {'train': 0.3151524331834581, 'test': 0.45623327046632767},
 {'train': 0.13287244157658684, 

In [5]:
pd.DataFrame(results).to_csv(f"results\\pure_LSTM_h{horizon}.csv", index=False)

# Augmentation with QuantGAN synthetic data

In [4]:
class CombinedDataLoader:
    def __init__(self, *dls):
        self.dls = dls

    def __len__(self):
        return sum(map(len, self.dls))
    
    def __iter__(self):
        for dl in self.dls:
            for v in dl:
                yield v

def train_synth(synthetic_path):
    ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected.csv")
    epochs = 2

    results = []
    for ts_index, time_series in tqdm(enumerate(ts_iterator)):
        synth_time_series = np.load(f"{synthetic_path}selected{ts_index}.npy")
        scaler = DimUniversalStandardScaler()
        synth_time_series = scaler.fit_transform(synth_time_series)
        synth_dls = []
        for i in range(synth_time_series.shape[0]):
            synth_dl, _, _, X_scaler, y_scaler = create_ts_dl(synth_time_series[i].reshape(- 1, 1), synth_time_series[i].flatten(), lags=lags, horizon=horizon, stride=stride,\
                                                batch_size=batch_size, device=device, data_preprocess=(None,),\
                                                val_size=0, test_size=0, drop_last=drop_last)
            synth_dls.append(synth_dl)

        train_dl, _, test_dl, *_ = create_ts_dl(time_series[["Close"]], time_series["Close"], lags=lags, horizon=horizon, stride=stride,\
                                                data_preprocess=("log_returns", "normalize"), device=device,\
                                                val_size=val_size, test_size=test_size, batch_size=batch_size, drop_last=drop_last, scaler=scaler)
        
        model = RNNModel(seed=0, device=device)
        model.set_model(SimpleLSTM, **model_params)
        optim_params = {'params': model.model.parameters(), 'lr': 4e-4}
        model.set_optim(torch.optim.AdamW, **optim_params)
        model.set_criterion(MAE)

        # cdl = CombinedDataLoader(train_dl, *synth_dls)
        # only synth data
        cdl = CombinedDataLoader(*synth_dls)
        model.train(cdl, epochs=epochs, print_info=verbose, keep_hs=keep_hs, agg_loss="mean")
        results.append({"train": model.eval(train_dl, agg_loss="mean"), "test": model.eval(test_dl, agg_loss="mean")})

        del model, train_dl, test_dl, cdl, synth_time_series, synth_dls
        torch.cuda.empty_cache()
    return results

In [7]:
results = train_synth(synthetic_path = f"{dataset_path}synthetic/QuantGAN/")
results

0it [00:00, ?it/s]Global seed set to 0
1it [00:35, 35.98s/it]Global seed set to 0
2it [01:16, 38.40s/it]Global seed set to 0
3it [01:48, 35.86s/it]Global seed set to 0
4it [02:27, 37.02s/it]Global seed set to 0
5it [03:07, 37.94s/it]Global seed set to 0
6it [03:54, 41.23s/it]Global seed set to 0
7it [04:40, 42.54s/it]Global seed set to 0
8it [05:21, 42.03s/it]Global seed set to 0
9it [06:04, 42.45s/it]Global seed set to 0
10it [06:46, 42.47s/it]Global seed set to 0
11it [07:19, 39.48s/it]Global seed set to 0
12it [08:04, 41.24s/it]Global seed set to 0
13it [08:48, 41.95s/it]Global seed set to 0
14it [09:33, 42.89s/it]Global seed set to 0
15it [10:18, 43.46s/it]Global seed set to 0
16it [11:01, 43.43s/it]Global seed set to 0
17it [11:46, 43.71s/it]Global seed set to 0
18it [12:35, 45.37s/it]Global seed set to 0
19it [13:21, 45.47s/it]Global seed set to 0
20it [14:08, 46.05s/it]Global seed set to 0
21it [14:44, 42.93s/it]Global seed set to 0
22it [15:30, 43.88s/it]Global seed set to 0
23

[{'train': 0.5399997681379318, 'test': 0.5369690656661987},
 {'train': 0.8218766252199808, 'test': 0.929135650396347},
 {'train': 0.7037610610326132, 'test': 0.5645192265510559},
 {'train': 0.7239434123039246, 'test': 0.6051509976387024},
 {'train': 0.6056286096572876, 'test': 0.45083755254745483},
 {'train': 0.7249199271202087, 'test': 0.7752843499183655},
 {'train': 0.7686007767915726, 'test': 1.0275514721870422},
 {'train': 0.660145714879036, 'test': 0.5406306385993958},
 {'train': 0.7586912989616394, 'test': 0.6183743079503378},
 {'train': 0.7447809875011444, 'test': 0.7281526327133179},
 {'train': 0.5368947796523571, 'test': 0.507084921002388},
 {'train': 0.6225552558898926, 'test': 0.43615976721048355},
 {'train': 0.6306326277554035, 'test': 0.5939633026719093},
 {'train': 0.6472753116062709, 'test': 0.43623635172843933},
 {'train': 0.6695150695741177, 'test': 0.4412989467382431},
 {'train': 0.635792420970069, 'test': 0.39255113154649734},
 {'train': 0.6511884894635942, 'test': 0

In [8]:
pd.DataFrame(results).to_csv(f"results\\QuantGAN_synth_LSTM_h{horizon}.csv", index=False)

# Augmentation with FourierFlow synthetic data

In [9]:
results = train_synth(synthetic_path = f"{dataset_path}synthetic/FourierFlow/")
results

0it [00:00, ?it/s]Global seed set to 0
1it [00:51, 51.11s/it]Global seed set to 0
2it [01:40, 50.00s/it]Global seed set to 0
3it [02:29, 49.48s/it]Global seed set to 0
4it [03:13, 47.30s/it]Global seed set to 0
5it [03:59, 46.94s/it]Global seed set to 0
6it [04:47, 47.33s/it]Global seed set to 0
7it [05:35, 47.65s/it]Global seed set to 0
8it [06:20, 46.83s/it]Global seed set to 0
9it [07:07, 46.76s/it]Global seed set to 0
10it [07:53, 46.44s/it]Global seed set to 0
11it [08:42, 47.26s/it]Global seed set to 0
12it [09:29, 47.35s/it]Global seed set to 0
13it [10:18, 47.66s/it]Global seed set to 0
14it [11:09, 48.84s/it]Global seed set to 0
15it [11:58, 48.68s/it]Global seed set to 0
16it [12:46, 48.47s/it]Global seed set to 0
17it [13:30, 47.30s/it]Global seed set to 0
18it [14:15, 46.67s/it]Global seed set to 0
19it [15:03, 46.90s/it]Global seed set to 0
20it [15:54, 48.28s/it]Global seed set to 0
21it [16:47, 49.67s/it]Global seed set to 0
22it [17:39, 50.28s/it]Global seed set to 0
23

[{'train': 0.2839076891541481, 'test': 0.4841673970222473},
 {'train': 0.3207463522752126, 'test': 0.7362737208604813},
 {'train': 0.2643931706746419, 'test': 0.6461139917373657},
 {'train': 0.20063621550798416, 'test': 0.7239977121353149},
 {'train': 0.18939536809921265, 'test': 0.5892677903175354},
 {'train': 0.40863676071166993, 'test': 0.7147764265537262},
 {'train': 0.41374319791793823, 'test': 0.9185154438018799},
 {'train': 0.34632231295108795, 'test': 0.5077514201402664},
 {'train': 0.3842129111289978, 'test': 0.58975750207901},
 {'train': 0.4036954790353775, 'test': 0.7789065539836884},
 {'train': 0.3560752607882023, 'test': 0.6422428414225578},
 {'train': 0.3999287448823452, 'test': 0.4340653344988823},
 {'train': 0.3899582251906395, 'test': 0.5847790017724037},
 {'train': 0.4678953928606851, 'test': 0.44919755061467487},
 {'train': 0.45767292380332947, 'test': 0.4320671260356903},
 {'train': 0.48379743761486477, 'test': 0.4032767713069916},
 {'train': 0.515582905875312, 'tes

In [10]:
pd.DataFrame(results).to_csv(f"results\\FourierFlow_synth_LSTM_h{horizon}.csv", index=False)

# Augmentation with RealNVP synthetic data

In [5]:
results = train_synth(synthetic_path = f"{dataset_path}synthetic/RealNVP/")
results

0it [00:00, ?it/s]Global seed set to 0
1it [00:53, 53.97s/it]Global seed set to 0
2it [01:38, 48.45s/it]Global seed set to 0
3it [02:24, 47.19s/it]Global seed set to 0
4it [03:06, 45.41s/it]Global seed set to 0
5it [03:53, 45.97s/it]Global seed set to 0
6it [04:39, 45.81s/it]Global seed set to 0
7it [05:28, 46.80s/it]Global seed set to 0
8it [06:13, 46.32s/it]Global seed set to 0
9it [06:59, 46.14s/it]Global seed set to 0
10it [07:44, 45.94s/it]Global seed set to 0
11it [08:33, 46.68s/it]Global seed set to 0
12it [09:19, 46.61s/it]Global seed set to 0
13it [10:06, 46.64s/it]Global seed set to 0
14it [10:52, 46.62s/it]Global seed set to 0
15it [11:38, 46.26s/it]Global seed set to 0
16it [12:25, 46.45s/it]Global seed set to 0
17it [13:11, 46.29s/it]Global seed set to 0
18it [13:57, 46.26s/it]Global seed set to 0
19it [14:43, 46.21s/it]Global seed set to 0
20it [15:32, 47.20s/it]Global seed set to 0
21it [16:22, 47.97s/it]Global seed set to 0
22it [17:10, 47.96s/it]Global seed set to 0
23

[{'train': 0.15004383772611618, 'test': 0.5828879475593567},
 {'train': 0.11478640884160995, 'test': 0.7357212007045746},
 {'train': 0.1276481474439303, 'test': 0.6526840925216675},
 {'train': 0.10549279674887657, 'test': 0.6811903715133667},
 {'train': 0.09763499721884727, 'test': 0.46719008684158325},
 {'train': 0.10917669087648392, 'test': 0.8167841136455536},
 {'train': 0.12991096451878548, 'test': 0.9803167283535004},
 {'train': 0.11692316643893719, 'test': 0.5921558737754822},
 {'train': 0.12485477775335312, 'test': 0.672295073668162},
 {'train': 0.11501871421933174, 'test': 0.797964870929718},
 {'train': 0.1739568766206503, 'test': 0.7106724455952644},
 {'train': 0.2287814673036337, 'test': 0.47544272243976593},
 {'train': 0.21562455408275127, 'test': 0.6364595144987106},
 {'train': 0.26897419137614115, 'test': 0.49685638149579364},
 {'train': 0.2757358383387327, 'test': 0.5303112044930458},
 {'train': 0.4087969164053599, 'test': 0.42675119638442993},
 {'train': 0.22328211698267

In [6]:
pd.DataFrame(results).to_csv(f"results\\RealNVP_synth_LSTM_h{horizon}.csv", index=False)

# Augmentation with TTS GAN synthetic data

In [7]:
results = train_synth(synthetic_path = f"{dataset_path}synthetic/TTS_GAN_standard/")
results

0it [00:00, ?it/s]Global seed set to 0
1it [00:41, 41.07s/it]Global seed set to 0
2it [01:19, 39.80s/it]Global seed set to 0
3it [01:58, 39.40s/it]Global seed set to 0
4it [02:37, 39.05s/it]Global seed set to 0
5it [03:16, 38.90s/it]Global seed set to 0
6it [03:54, 38.84s/it]Global seed set to 0
7it [04:33, 38.87s/it]Global seed set to 0
8it [05:21, 41.57s/it]Global seed set to 0
9it [06:02, 41.55s/it]Global seed set to 0
10it [06:43, 41.48s/it]Global seed set to 0
11it [07:25, 41.53s/it]Global seed set to 0
12it [08:07, 41.69s/it]Global seed set to 0
13it [08:47, 41.29s/it]Global seed set to 0
14it [09:27, 40.77s/it]Global seed set to 0
15it [10:07, 40.60s/it]Global seed set to 0
16it [10:53, 42.09s/it]Global seed set to 0
17it [11:31, 40.79s/it]Global seed set to 0
18it [12:08, 39.80s/it]Global seed set to 0
19it [12:45, 38.93s/it]Global seed set to 0
20it [13:22, 38.36s/it]Global seed set to 0
21it [14:00, 38.39s/it]Global seed set to 0
22it [14:39, 38.35s/it]Global seed set to 0
23

[{'train': 0.3464639335870743, 'test': 0.3495182394981384},
 {'train': 0.47238794962565106, 'test': 0.5490506589412689},
 {'train': 0.13970942795276642, 'test': 0.14457552134990692},
 {'train': 0.10603639855980873, 'test': 0.0863032341003418},
 {'train': 0.24075046181678772, 'test': 0.23972749710083008},
 {'train': 0.7824883460998535, 'test': 0.8433496952056885},
 {'train': 0.5399437099695206, 'test': 0.7368926405906677},
 {'train': 0.7483021169900894, 'test': 0.5924322605133057},
 {'train': 0.2259869694709778, 'test': 0.18373537560304007},
 {'train': 0.22070566564798355, 'test': 0.21476686000823975},
 {'train': 1.0536351278424263, 'test': 0.9987778961658478},
 {'train': 0.9251511916518211, 'test': 0.6740890741348267},
 {'train': 0.7261753231287003, 'test': 0.6697108745574951},
 {'train': 0.5284964314528874, 'test': 0.35438694556554157},
 {'train': 0.8310432285070419, 'test': 0.5499319061636925},
 {'train': 3.595274395412869, 'test': 3.606085956096649},
 {'train': 0.7920023136668735, '

In [8]:
pd.DataFrame(results).to_csv(f"results\\TTS_GAN_synth_LSTM_h{horizon}.csv", index=False)

# Comparing quality on real & synthetic data

# Testing on real data

In [4]:
import numpy as np
import pandas as pd

from tqdm import tqdm

from utils.data import *
from utils.metrics import MAPE, WAPE, MAE
from utils.dl import *

hsm_dataset_dir = Path("data/huge_stock_market_dataset/")
se_dataset_dir = Path("data/solar_energy/")
results_dir = Path("results")

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

lags = 32
horizon = 8
stride = 1
batch_size = 256
test_size = 0.3
drop_last = False
features = 1
keep_hs = False  # stride == lags
verbose = False

model_params = {'input_size': features, 'hidden_size': 256, 'num_layers': 2, 'dropout': 0.1, 'output_size': horizon, 'seq_len': lags}

cuda:0


In [11]:
start_point = 0
start_ds = 1

for dataset_path, dataset_name in ((Path("data/huge_stock_market_dataset/"), "hsm"),\
     (Path("data/solar_energy"), "se")):
    if dataset_name == "hsm" and start_ds == 1: continue
    if dataset_name == "hsm":
        ts_iterator = get_hsm_dataset(dataset_path, selected_files=f"{dataset_path}/selected100.csv")
        epochs = 40
    else:
        ts_iterator = get_solar_energy_dataset(dataset_path, max_results=10)
        epochs = 5
    for _ in range(start_point): next(ts_iterator)

    target_col = "Close" if dataset_name == "hsm" else "Power(MW)"
    results = []
    for time_series in tqdm(ts_iterator):
        if dataset_name == "se": time_series += 1e-9
        train_dl, _, test_dl, X_scaler, y_scaler = create_ts_dl(time_series[[target_col]], time_series[target_col], lags=lags, horizon=horizon, stride=stride,\
                                                batch_size=batch_size, device=device, data_preprocess=("log_returns", "normalize"),\
                                                val_size=0, test_size=test_size, drop_last=drop_last)
        
        model = RNNModel(seed=0, device=device)
        model.set_model(SimpleLSTM, **model_params)
        optim_params = {'params': model.model.parameters(), 'lr': 4e-4}
        model.set_optim(torch.optim.AdamW, **optim_params)
        model.set_criterion(MAE)

        model.train(train_dl, epochs=epochs, print_info=verbose, keep_hs=keep_hs, agg_loss="mean")
        results.append(model.eval(test_dl, agg_loss="mean"))

        del model, train_dl, test_dl
        torch.cuda.empty_cache()
    pd.DataFrame(results, columns=["test"]).to_csv(results_dir / f"real_{dataset_name}_LSTM.csv", index=False)

0it [00:00, ?it/s]Global seed set to 0
1it [00:49, 49.06s/it]Global seed set to 0
2it [01:36, 47.87s/it]Global seed set to 0
3it [02:22, 47.18s/it]Global seed set to 0
4it [03:09, 47.17s/it]Global seed set to 0
5it [03:56, 47.04s/it]Global seed set to 0
6it [04:43, 46.90s/it]Global seed set to 0
7it [05:30, 46.92s/it]Global seed set to 0
8it [06:16, 46.75s/it]Global seed set to 0
9it [07:03, 46.87s/it]Global seed set to 0
10it [07:50, 47.02s/it]


# Testing on synthetic data

In [13]:
start_synth_model = 0
for synth_model_ind, synth_model in enumerate(("QuantGAN", "RealNVP", "FourierFlow", "TTS_GAN")):
    if synth_model_ind < start_synth_model: continue
    epochs_synth = 5
    for dataset_path, dataset_name in ((Path("data/huge_stock_market_dataset/synthetic/"), "hsm"),\
     (Path("data/solar_energy/synthetic/"), "se")):
        synth_path = dataset_path / synth_model
        results = []
        print(f"Testing synthetic data from {synth_model} on {dataset_name} dataset")
        for ts_index in tqdm(range(100 if dataset_name == "hsm" else 10)):
            synth_time_series = np.load(synth_path / f"selected{ts_index}.npy")
            results.append(0)
            num_synth_samples = min(10, synth_time_series.shape[0]) if synth_model in ("QuantGAN", "TTS_GAN") else 2
            for i in range(num_synth_samples):
                train_dl, _, test_dl, X_scaler, y_scaler = create_ts_dl(synth_time_series[i].reshape(- 1, 1), synth_time_series[i].flatten(), lags=lags, horizon=horizon, stride=stride,\
                                                    batch_size=batch_size, device=device, data_preprocess=("normalize",),\
                                                    val_size=0, test_size=0.3, drop_last=drop_last)
                model = RNNModel(seed=0, device=device)
                model.set_model(SimpleLSTM, **model_params)
                optim_params = {'params': model.model.parameters(), 'lr': 4e-4}
                model.set_optim(torch.optim.AdamW, **optim_params)
                model.set_criterion(MAE)
                model.train(train_dl, epochs=epochs_synth, print_info=verbose, keep_hs=keep_hs, agg_loss="mean")
                results[- 1] += model.eval(test_dl, agg_loss="mean")
                del model, train_dl, test_dl
                torch.cuda.empty_cache()
            results[- 1] /= num_synth_samples
            del synth_time_series
        pd.DataFrame(results, columns=["test"]).to_csv(results_dir / f"synth_{synth_model}_{dataset_name}_LSTM.csv", index=False)

Testing synthetic data from QuantGAN on hsm dataset


  0%|          | 0/100 [00:00<?, ?it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
  1%|          | 1/100 [00:00<01:09,  1.42it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
  2%|▏         | 2/100 [00:01<00:59,  1.66it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
  3%|▎         | 3/100 [00:01<00:55,  1.75it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
G

Testing synthetic data from QuantGAN on se dataset


  0%|          | 0/10 [00:00<?, ?it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
 10%|█         | 1/10 [00:00<00:04,  1.89it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
 20%|██        | 2/10 [00:01<00:04,  1.86it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
 30%|███       | 3/10 [00:01<00:03,  1.81it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Globa

Testing synthetic data from RealNVP on hsm dataset


...

  0%|          | 0/10 [00:00<?, ?it/s]Global seed set to 0
Global seed set to 0
 10%|█         | 1/10 [00:07<01:03,  7.11s/it]Global seed set to 0
Global seed set to 0
 20%|██        | 2/10 [00:14<00:57,  7.22s/it]Global seed set to 0
Global seed set to 0
 30%|███       | 3/10 [00:21<00:50,  7.15s/it]Global seed set to 0
Global seed set to 0
 40%|████      | 4/10 [00:28<00:43,  7.24s/it]Global seed set to 0
Global seed set to 0
 50%|█████     | 5/10 [00:35<00:35,  7.17s/it]Global seed set to 0
Global seed set to 0
 60%|██████    | 6/10 [00:43<00:28,  7.15s/it]Global seed set to 0
Global seed set to 0
 70%|███████   | 7/10 [00:50<00:21,  7.22s/it]Global seed set to 0
Global seed set to 0
 80%|████████  | 8/10 [00:57<00:14,  7.24s/it]Global seed set to 0
Global seed set to 0
 90%|█████████ | 9/10 [01:04<00:07,  7.22s/it]Global seed set to 0
Global seed set to 0
100%|██████████| 10/10 [01:12<00:00,  7.22s/it]


Testing synthetic data from TTS_GAN on hsm dataset


  0%|          | 0/100 [00:00<?, ?it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
  1%|          | 1/100 [00:00<01:16,  1.30it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
  2%|▏         | 2/100 [00:01<01:16,  1.28it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
  3%|▎         | 3/100 [00:02<01:17,  1.25it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
G

Testing synthetic data from TTS_GAN on se dataset


  0%|          | 0/10 [00:00<?, ?it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
 10%|█         | 1/10 [00:00<00:07,  1.28it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
 20%|██        | 2/10 [00:01<00:06,  1.32it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
 30%|███       | 3/10 [00:02<00:05,  1.33it/s]Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Global seed set to 0
Globa